In [ ]:
%pip install duckduckgo_search openai loguru

In [36]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(region="cn-zh", time="d", max_results=5)
search = DuckDuckGoSearchResults(api_wrapper=wrapper)
results = search.invoke("elon musk wife")
print(results)

snippet: Elon Musk, the visionary entrepreneur behind Tesla and SpaceX, is not just known for his groundbreaking ideas but also for his complex personal life. At the center of this intriguing narrative is his relationship with musician Claire Boucher, better known by her stage name, Grimes. Their romance has captured the attention of fans and media ..., title: Elon Musk's Life Partner: A Deep Dive Into His Relationship With Grimes, link: https://jenkins-ci.syncfusion.com/travel-guides10/elon-musk-wife.html, snippet: The charming outing, held at one of Trump's courses, turned out to be a blend of family and familiar faces, as tech mogul Elon Musk also made a surprise appearance with his son, X Æ A-12. The ..., title: Kai Trump, 17, enjoys special day with grandpa Donald and 'uncle' Elon Musk, link: https://www.hellomagazine.com/celebrities/729223/kai-trump-17-enjoys-special-day-with-grandpa-donald-and-uncle-elon-musk/, snippet: From reproductive rights to climate change to Big Tech, The

In [34]:
import re
import json
import requests
from typing import List, Dict, Any, Optional
from loguru import logger
from openai import OpenAI

# Configure loguru
logger.remove()
logger.add(
    "rewoo_{time}.log",
    format="<green>{time:YYYY-MM-DD HH:mm:ss}</green> | <level>{level}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - <level>{message}</level>",
    level="DEBUG",
    rotation="1 day"
)
logger.add(
    lambda msg: print(msg),
    colorize=True,
    format="<green>{time:HH:mm:ss}</green> | <level>{level}</level> | <level>{message}</level>"
)

class ReWOO:
    def __init__(self, openai_api_key: str, base_url: str, model:str, search_api_key: str):
        logger.info("Initializing ReWOO")
        self.openai_api_key = openai_api_key
        self.base_url = base_url
        self.model = model
        self.search_api_key = search_api_key

    def call_openai_api(self, prompt: str) -> str:
        """Simulates OpenAI API call"""
        logger.debug(f"Calling OpenAI API with prompt: {prompt}...")
        try:
            client = OpenAI(
                api_key=self.openai_api_key,
                base_url=self.base_url,
            )

            completion = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "user", "content": prompt}
                ],
            )
            response = completion.choices[0].message.content
            logger.debug(f"OpenAI API response: {response}...")
            return response
        except Exception as e:
            logger.error(f"Error calling OpenAI API: {str(e)}")
            raise

    def search_web(self, query: str) -> str:
        """Simulates web search API call"""
        logger.debug(f"Performing web search with query: {query}")
        try:
            from langchain_community.tools import DuckDuckGoSearchRun
            search = DuckDuckGoSearchRun()
            result = search.invoke(query)
            logger.debug(f"Search results: {result}...")
            return result
        except Exception as e:
            logger.error(f"Error in web search: {str(e)}")
            raise

    def create_plan(self, task: str) -> Dict[str, Any]:
        """Creates execution plan for the given task"""
        logger.info(f"Creating plan for task: {task}")
        prompt = f"""
        For the following task, make plans that can solve the problem step by step.
        For each plan, indicate which external tool together with tool input to retrieve evidence.
        You can store the evidence into a variable #E that can be called by later tools.

        Tools available:
        1. Google[input]: Searches the web. Input should be a search query.
        2. LLM[input]: Uses AI for reasoning. Input can be any instruction.

        Each plan should be in format:
        Plan: <reasoning>
        #E<number> = Tool[argument]

        Task: {task}
        """

        response = self.call_openai_api(prompt)

        # Parse plans using regex
        pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
        steps = re.findall(pattern, response)
        logger.debug(f"Created plan with {len(steps)} steps")

        return {
            "plan_string": response,
            "steps": steps
        }

    def execute_step(self, tool: str, input_str: str, variables: Dict[str, str]) -> str:
        """Executes a single step in the plan"""
        logger.info(f"Executing step with tool: {tool}")
        # Replace variables in input
        for var, value in variables.items():
            input_str = input_str.replace(var, value)

        logger.debug(f"Processed input: {input_str}...")

        try:
            if tool.lower() == "google":
                results = self.search_web(input_str)
                return json.dumps(results)
            elif tool.lower() == "llm":
                return self.call_openai_api(input_str)
            else:
                raise ValueError(f"Unknown tool: {tool}")
        except Exception as e:
            logger.error(f"Error executing step: {str(e)}")
            raise

    def solve(self, task: str) -> str:
        """Main solving logic"""
        logger.info(f"Starting to solve task: {task}")
        try:
            # Create plan
            plan = self.create_plan(task)
            results = {}

            # Execute each step
            for idx, (reasoning, var_name, tool, tool_input) in enumerate(plan["steps"], 1):
                logger.info(f"Executing step {idx}/{len(plan['steps'])}")
                logger.debug(f"Reasoning: {reasoning}")
                logger.debug(f"Variable: {var_name}")
                logger.debug(f"Tool: {tool}")
                logger.debug(f"Input: {tool_input}")

                result = self.execute_step(tool, tool_input, results)
                results[var_name] = result

            # Generate final answer
            logger.info("Generating final answer")
            solve_prompt = f"""
            Solve the following task using the evidence collected:

            Task: {task}

            Evidence:
            {json.dumps(results, indent=2)}

            Provide a direct answer with no extra words.
            """

            final_answer = self.call_openai_api(solve_prompt)
            logger.success(f"Task completed. Answer: {final_answer}")
            return final_answer

        except Exception as e:
            logger.error(f"Error solving task: {str(e)}")
            raise

class State:
    def __init__(self, task: str):
        self.task = task
        self.plan_string: str = ""
        self.steps: List = []
        self.results: Dict[str, str] = {}
        self.result: str = ""

def main():
    logger.info("Starting ReWOO application")
    try:
        # Initialize with API keys
        rewoo = ReWOO(
            openai_api_key="xai-eUy22SMHpaMbGM5M5kHqAmcQVCYxKEqlvWBJCgugk3oy5Lqay7ZHFSJmKu4Hu93otjmMNUttXICadq",
            base_url="https://api.x.ai/v1",
            model="grok-beta",
            search_api_key="21622232a6756fe9881da641506e2ba3d46287c85dffee2acedec1a3fb96a8"
        )

        # Example usage
        task = "Will Elon Mask take us to Mars?"
        logger.info(f"Processing task: {task}")

        answer = rewoo.solve(task)
        logger.success(f"Task completed successfully")
        logger.info(f"Task: {task}")
        logger.info(f"Answer: {answer}")

    except Exception as e:
        logger.error(f"Application error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

08:07:16 | INFO | Starting ReWOO application

08:07:16 | INFO | Initializing ReWOO

08:07:16 | INFO | Processing task: Will Elon Mask take us to Mars?

08:07:16 | INFO | Starting to solve task: Will Elon Mask take us to Mars?

08:07:16 | INFO | Creating plan for task: Will Elon Mask take us to Mars?

08:07:16 | DEBUG | Calling OpenAI API with prompt: 
        For the following task, make plans that can solve the problem step by step. 
        For each plan, indicate which external tool together with tool input to retrieve evidence.
        You can store the evidence into a variable #E that can be called by later tools.
        
        Tools available:
        1. Google[input]: Searches the web. Input should be a search query.
        2. LLM[input]: Uses AI for reasoning. Input can be any instruction.
        
        Each plan should be in format:
        Plan: <reasoning>
        #E<number> = Tool[argument]
        
        Task: Will Elon Mask take us to Mars?
        ...

08:07:25 